In [ ]:
using Pkg
Pkg.activate("../Tambo/")
using Tambo
using PyCall
using Glob
using JLD2
using StatsBase
using Plots


In [ ]:
include("../figures/paperstyle.jl");

# Define some constants

In [ ]:
np = pyimport("numpy")

const zmin = -1100units.m
const zmax = 1100units.m

In [ ]:
sim = jldopen("/Users/jlazar/Downloads/injection_20231227.jld2")
events = sim["injected_events"]

filter(x->x[1]!=:geo_spline_path, sim["config"])
config = SimulationConfig(; geo_spline_path="../resources/tambo_spline.jld2", filter(x->x[1]!=:geo_spline_path, sim["config"])...)
geo = Tambo.Geometry(config)
plane = Tambo.Plane(whitepaper_normal_vec, whitepaper_coord, geo)
injector = Tambo.Injector(config);

# Make the flux models

In [ ]:
flux_models = Dict()

γ = 2.37
norm = 1.44e-18 / units.GeV / units.cm^2 / units.second * (1 /(100units.TeV))^-γ
pl = Tambo.PowerLaw(γ, 100units.GeV, 1e9units.GeV, norm)

flux_models["hese"] = pl

# Total number of events vs. number of modules

In [ ]:
plt = plot(
    xscale=:log,
    yscale=:log,
    ylabel=L"N_{\mathrm{evts}}~\left[\mathrm{yr}^{-1}\right]",
    xlabel=L"N_{\mathrm{mods}}",
    bottommargin=2mm,
    leftmargin=2mm,
    # ylim=(0, 0.13)
)


for Δs in [100, 150, 200]
    fs = glob("*_$(Δs).npy", "./output/")
    sort!(fs, by=f -> ℓ = parse(Float64, split(split(f, "_")[1], "/")[end]))

    ns = []
    rates = []

    for f in fs
        event_idxs = np.load(f)
        evts = events[event_idxs]
        ℓ = parse(Float64, split(split(f, "_")[1], "/")[end]) * units.m

        fluxes = flux_models["hese"].(evts["initial_state"]["energy"])

        weights = Tambo.oneweight.(
            evts,
            Ref(injector.xs),
            Ref(injector.xs),
            Ref(injector.powerlaw),
            Ref(injector.anglesampler),
            Ref(injector.injectionvolume),
            Ref(geo)
        ) / maximum(event_idxs)

        push!(rates, sum(fluxes .* weights .* units.second * 10^7.5))

        detection_modules = Tambo.make_trianglearray(-2000units.m, 3000units.m, -ℓ/2, ℓ/2, Δs * units.m, ϕ=whitepaper_normal_vec.ϕ)
        mask = zmin .< Tambo.plane_z.(getfield.(detection_modules, :x), getfield.(detection_modules, :y), Ref(plane)) .< zmax;
        push!(ns, sum(mask))
    end

    plot!(plt, ns, rates, label=L"\Delta s = %$(Δs)~\mathrm{m}")
    scatter!(plt, ns, rates)
    
    
    
    
end
savefig(plt, "../figures/nevents_vs_nmods.pdf")
display(plt)

In [ ]:
plt = plot(
    xscale=:log,
    yscale=:log,
    ylabel=L"N_{\mathrm{evts}}~\left[\mathrm{yr}^{-1}\right]",
    xlabel=L"N_{\mathrm{mods}}",
    bottommargin=2mm,
    leftmargin=2mm,
    # ylim=(0, 0.13)
)


for Δs in [100, 150, 200]
    fs = glob("*_$(Δs).npy", "./output/")
    sort!(fs, by=f -> ℓ = parse(Float64, split(split(f, "_")[1], "/")[end]))

    ns = []
    rates = []

    for f in fs
        event_idxs = np.load(f)
        evts = events[event_idxs]
        ℓ = parse(Float64, split(split(f, "_")[1], "/")[end]) * units.m

        fluxes = flux_models["hese"].(evts["initial_state"]["energy"])

        weights = Tambo.oneweight.(
            evts,
            Ref(injector.xs),
            Ref(injector.xs),
            Ref(injector.powerlaw),
            Ref(injector.anglesampler),
            Ref(injector.injectionvolume),
            Ref(geo)
        ) / maximum(event_idxs)

        push!(rates, sum(fluxes .* weights .* units.second * 10^7.5))

        detection_modules = Tambo.make_trianglearray(-2000units.m, 3000units.m, -ℓ/2, ℓ/2, Δs * units.m, ϕ=whitepaper_normal_vec.ϕ)
        mask = zmin .< Tambo.plane_z.(getfield.(detection_modules, :x), getfield.(detection_modules, :y), Ref(plane)) .< zmax;
        push!(ns, sum(mask))
    end

    plot!(plt, ns, rates ./ ns, label=L"\Delta s = %$(Δs)~\mathrm{m}")
    scatter!(plt, ns, rates./ns)
end
# savefig(plt, "../figures/nevents_vs_nmods.pdf")
display(plt)

# Number of events versus $\ell$

In [ ]:
plt = plot(
    xscale=:log,
    yscale=:log,
    ylabel=L"N_{\mathrm{evts}}~\left[\mathrm{yr}^{-1}\right]",
    xlabel=L"\ell_{\mathrm{det.}}~\left[\mathrm{km}\right]",
    bottommargin=2mm,
    leftmargin=2mm,
    # ylim=(0, 0.13)
)


for Δs in [100, 150, 200]
    fs = glob("*_$(Δs).npy", "./output/")
    sort!(fs, by=f -> ℓ = parse(Float64, split(split(f, "_")[1], "/")[end]))

    ℓs = []
    rates = []

    for f in fs
        event_idxs = np.load(f)
        evts = events[event_idxs]
        ℓ = parse(Float64, split(split(f, "_")[1], "/")[end]) * units.m

        fluxes = flux_models["hese"].(evts["initial_state"]["energy"])

        weights = Tambo.oneweight.(
            evts,
            Ref(injector.xs),
            Ref(injector.xs),
            Ref(injector.powerlaw),
            Ref(injector.anglesampler),
            Ref(injector.injectionvolume),
            Ref(geo)
        ) / maximum(event_idxs)

        push!(rates, sum(fluxes .* weights .* units.second * 10^7.5))

        # detection_modules = Tambo.make_trianglearray(-2000units.m, 3000units.m, -ℓ/2, ℓ/2, Δs * units.m, ϕ=whitepaper_normal_vec.ϕ)
        # mask = zmin .< Tambo.plane_z.(getfield.(detection_modules, :x), getfield.(detection_modules, :y), Ref(plane)) .< zmax;
        push!(ℓs, ℓ)
    end

    plot!(plt, ℓs ./ units.km, rates, label=L"\Delta s = %$(Δs)~\mathrm{m}")
    scatter!(plt, ℓs ./ units.km, rates)
end
# savefig(plt, "../figures/nevents_vs_length.pdf")
display(plt)



In [ ]:
Δs = 150

good_numbers = [6, 7, 8]

plt = plot(
    xscale=:log,
    yscale=:log,
    xlabel=L"E_{\nu}~\left[\mathrm{GeV}\right]",
    ylabel=L"E_{\nu}\frac{\mathrm{d}N_{\nu}}{\mathrm{d}E_{\nu}\mathrm{d}t}~\left[\mathrm{yr}^{-1}\right]",
    bottommargin=5mm,
    leftmargin=5mm,
    xlimits=(1e6, 1e9),
    ylimits=(4e-7, 1e-4),
    size=(600, 400),
    legend_columns=3,
)

# Load up all the files
fs = glob("*_$(Δs).npy", "./output/")
sort!(fs, by=f -> parse(Float64, split(split(f, "_")[1], "/")[end]))

edges = 10 .^ (6:0.25:9) * units.GeV

colors = cgrad(:matter, length(fs), categorical = true)

for (jdx, f) in enumerate(fs)


    event_idxs = np.load(f)
    evts = events[event_idxs]
    ℓ = parse(Float64, split(split(f, "_")[1], "/")[end]) * units.m

    fluxes = flux_models["hese"].(evts["initial_state"]["energy"])

    weights = Tambo.oneweight.(
        evts,
        Ref(injector.xs),
        Ref(injector.xs),
        Ref(injector.powerlaw),
        Ref(injector.anglesampler),
        Ref(injector.injectionvolume),
        Ref(geo)
    ) / maximum(event_idxs)

    detection_modules = Tambo.make_trianglearray(-2000units.m, 3000units.m, -ℓ/2, ℓ/2, Δs * units.m, ϕ=whitepaper_normal_vec.ϕ)
    
    mask = zmin .< Tambo.plane_z.(getfield.(detection_modules, :x), getfield.(detection_modules, :y), Ref(plane)) .< zmax;

    hist = fit(Histogram, evts["initial_state"]["energy"], Weights(fluxes .* weights), edges)

    cents = (hist.edges[1][1:end-1] .+ hist.edges[1][2:end]) / 2
    widths = hist.edges[1][2:end] .- hist.edges[1][1:end-1]

    label = Int(ℓ / units.m)
    
    lw = 1
    if jdx in good_numbers
        lw = 2
    end

    plot!(
        plt, 
        cents / units.GeV,
        cents .* hist.weights ./ widths .* units.second * 10^7.5 / sum(mask),
        label=label,
        linetype=:stepmid,
        color=colors[jdx],
        linewidth=lw
    )

end
display(plt)


In [ ]:
Δs = 150

# Load up all the files
fs = glob("*_$(Δs).npy", "./output/")
sort!(fs, by=f -> parse(Float64, split(split(f, "_")[1], "/")[end]))

edges = 10 .^ (6:0.33333333:9) * units.GeV
cents = (edges[1:end-1] .+ edges[2:end]) / 2
widths = edges[2:end] .- edges[1:end-1]

colors = cgrad(:matter, length(fs), categorical = true)

A = Array{Float64, 2}(undef, length(fs), length(edges)-1)

nmods = []

for (jdx, f) in enumerate(fs)

    event_idxs = np.load(f)
    evts = events[event_idxs]
    ℓ = parse(Float64, split(split(f, "_")[1], "/")[end]) * units.m

    fluxes = flux_models["hese"].(evts["initial_state"]["energy"])

    weights = Tambo.oneweight.(
        evts,
        Ref(injector.xs),
        Ref(injector.xs),
        Ref(injector.powerlaw),
        Ref(injector.anglesampler),
        Ref(injector.injectionvolume),
        Ref(geo)
    ) / maximum(event_idxs)

    detection_modules = Tambo.make_trianglearray(-2000units.m, 3000units.m, -ℓ/2, ℓ/2, Δs * units.m, ϕ=whitepaper_normal_vec.ϕ)
    
    mask = zmin .< Tambo.plane_z.(getfield.(detection_modules, :x), getfield.(detection_modules, :y), Ref(plane)) .< zmax;

    hist = fit(Histogram, evts["initial_state"]["energy"], Weights(fluxes .* weights), edges)
    
    A[jdx, :] = hist.weights
    push!(nmods, sum(mask))

end

In [ ]:
plt = plot(size=(600, 550), rightmargin=1cm)

heatmap!(
    plt,
    log.(10, cents) .- 9,
    1:length(fs),
    log.(10, A),
    # clim=(-6, -4),
    colorbar_title=L"E_{\nu}\frac{\mathrm{d}N}{\mathrm{d}E_{\nu}}",
)

In [ ]:
funcs = []

for idx in 1:9

    plt = plot(
        xscale=:log,
        yscale=:log,
        # layout=(2,1),
        title="$(cents[idx] / units.GeV) GeV",
        xlabel=L"N_{\mathrm{mod}}",
        # ylabel=L"E_{\nu}\frac{\mathrm{d}N}{\mathrm{d}E_{\nu}}",
        leftmargin=2mm,
        bottommargin=2mm,
    )
    plot!(plt[1], nmods, A[:, idx])
    
    xs = log.(nmods[1:end-2])
    ys = log.(A[1:end-2, idx])
    mat = hcat(xs,ones(length(xs)))
    slope, intercept = mat \ ys
    println(slope)
    g(x) = slope*x + intercept
    push!(funcs, g)
    plot!(nmods, exp.(g.(log.(nmods))))
    
    display(plt)
end

In [ ]:
plt = plot(cents, [exp(f(log(22_000))) for f in funcs], xscale=:log, yscale=:log, linetype=:stepmid)

In [ ]:
sum([exp(f(log(6_000))) for f in funcs]) * units.second * 10^7.5

In [ ]:
event_idxs = np.load(f)
evts = events[event_idxs]
ℓ = parse(Float64, split(split(f, "_")[1], "/")[end]) * units.m

fluxes = flux_models["hese"].(evts["initial_state"]["energy"])

weights = Tambo.oneweight.(
    evts,
    Ref(injector.xs),
    Ref(injector.xs),
    Ref(injector.powerlaw),
    Ref(injector.anglesampler),
    Ref(injector.injectionvolume),
    Ref(geo)
) / maximum(event_idxs)

detection_modules = Tambo.make_trianglearray(-2000units.m, 3000units.m, -ℓ/2, ℓ/2, Δs * units.m, ϕ=whitepaper_normal_vec.ϕ)
mask = zmin .< Tambo.plane_z.(getfield.(detection_modules, :x), getfield.(detection_modules, :y), Ref(plane)) .< zmax
nmod = sum(mask)

hist = fit(Histogram, evts["initial_state"]["energy"], Weights(fluxes .* weights), edges)

cents = (hist.edges[1][1:end-1] .+ hist.edges[1][2:end]) / 2
widths = hist.edges[1][2:end] .- hist.edges[1][1:end-1]

label = Int(ℓ / units.m)

plt = plot(
    # plt, 
    cents / units.GeV,
    cents .* hist.weights ./ widths .* units.second * 10^7.5 / nmod * 22000,
    label="Naive scaling",
    linetype=:stepmid,
    # color=,
    linewidth=lw,
    xscale=:log,
    yscale=:log,  
)

plot!(plt, cents ./ units.GeV, [exp(f(log(22_000))) for f in funcs], xscale=:log, yscale=:log, linetype=:stepmid, label="Jeff scaling")

In [ ]:
plt = plot(
    xscale=:log,
    yscale=:log,
    ylabel=L"N_{\mathrm{evts}}~\left[\mathrm{yr}^{-1}\right]",
    xlabel=L"N_{\mathrm{mods}}",
    bottommargin=2mm,
    leftmargin=2mm,
    # ylim=(0, 0.13)
)


for Δs in [100, 150, 200]
    fs = glob("*_$(Δs).npy", "./output/")
    sort!(fs, by=f -> ℓ = parse(Float64, split(split(f, "_")[1], "/")[end]))

    ns = []
    rates = []

    for f in fs
        event_idxs = np.load(f)
        evts = events[event_idxs]
        ℓ = parse(Float64, split(split(f, "_")[1], "/")[end]) * units.m

        fluxes = flux_models["hese"].(evts["initial_state"]["energy"])

        weights = Tambo.oneweight.(
            evts,
            Ref(injector.xs),
            Ref(injector.xs),
            Ref(injector.powerlaw),
            Ref(injector.anglesampler),
            Ref(injector.injectionvolume),
            Ref(geo)
        ) / maximum(event_idxs)

        push!(rates, sum(fluxes .* weights .* units.second * 10^7.5))

        detection_modules = Tambo.make_trianglearray(-2000units.m, 3000units.m, -ℓ/2, ℓ/2, Δs * units.m, ϕ=whitepaper_normal_vec.ϕ)
        mask = zmin .< Tambo.plane_z.(getfield.(detection_modules, :x), getfield.(detection_modules, :y), Ref(plane)) .< zmax;
        push!(ns, sum(mask))
    end

    plot!(plt, ns, rates, label=L"\Delta s = %$(Δs)~\mathrm{m}")
    scatter!(plt, ns, rates)
    
    
    xs = log.(ns[2:end-2])
    ys = log.(rates[2:end-2])
    mat = hcat(xs, ones(length(xs)))
    slope, intercept = mat \ ys
    g(x) = slope*x + intercept
    
    println(slope)
    
    plot!(plt, ns, exp.(g.(log.(ns))))
    
    println((Δs, exp(g(log(22_000)))))
    
end
savefig(plt, "../figures/nevents_vs_nmods.pdf")
display(plt)